In [118]:
import openai
import pickle
import os
import time
import pandas as pd
save_dir = 'Data\\'
import statsmodels.api as sm
import json

In [132]:
case_dir = 'Data\\'
cases = pd.read_csv(case_dir +'Dyspnea cases.csv')

In [134]:
for c in range(4):
    DDx_real = cases.loc[c,'DDx'].replace('\n',' ')
    title = cases.loc[c,'title']
    Dyspnea = pickle.load(open(case_dir+'Dyspnea_case'+title+'_15.pkl', 'rb'))
    match_dict = dict()
    for d in demo_list:
        demo_json_list = list()
        with open(case_dir+'matching_prompt.txt', 'r') as file:
            base_prompt = file.read()
        for i in range(len(Dyspnea[d])):
            string = Dyspnea[d][i]
            try:
                cur_json = json.loads(string)
            except json.JSONDecodeError:
                try:
                    # Try adding missing characters and parse again
                    string += '}' if string.endswith(']') else ']}' if string.endswith('"') else '"]}'
                    cur_json = json.loads(string)
                except json.JSONDecodeError:
                    print(f"Unable to parse JSON: {string}")
            cur_keys = list(cur_json.keys())
            idx = [i for i in range(len(cur_keys)) if cur_keys[i].lower().find('differential diagnos')>-1]
            cur_ddx = cur_json[cur_keys[idx[0]]]
            list_two = ', '.join(cur_ddx)
            prompt = base_prompt + 'List One: [' + DDx_real +']' + '\n List Two: '+list_two
            response = openai.ChatCompletion.create(
                  engine="gpt-4",
                  messages=[
                    {"role": "user", "content": prompt}
                  ],temperature=0.7,max_tokens=500,
                )
            cur_str = response['choices'][0]['message']['content']
            st_json = cur_str.find('{')
            demo_json_list.append(json.loads(cur_str[st_json:]))
            print(d+' '+str(i))
        match_dict[d] = demo_json_list
    match_dict['Real_DDx'] = DDx_real
    with open(save_dir+'Dyspnea_case_'+str(c)+'_matched_DDx_15.pkl', 'wb') as file:
        pickle.dump(match_dict, file)

Female_Caucasian 0
Female_Caucasian 1
Female_Caucasian 2
Female_Caucasian 3
Female_Caucasian 4
Female_Caucasian 5
Female_Caucasian 6
Female_Caucasian 7
Female_Caucasian 8
Female_Caucasian 9
Female_Caucasian 10
Female_Caucasian 11
Female_Caucasian 12
Female_Caucasian 13
Female_Caucasian 14
Male_Caucasian 0
Male_Caucasian 1
Male_Caucasian 2
Male_Caucasian 3
Male_Caucasian 4
Male_Caucasian 5
Male_Caucasian 6
Male_Caucasian 7
Male_Caucasian 8
Male_Caucasian 9
Male_Caucasian 10
Male_Caucasian 11
Male_Caucasian 12
Male_Caucasian 13
Male_Caucasian 14
Female_Black 0
Female_Black 1
Female_Black 2
Female_Black 3
Female_Black 4
Female_Black 5
Female_Black 6
Female_Black 7
Female_Black 8
Female_Black 9
Female_Black 10
Female_Black 11
Female_Black 12
Female_Black 13
Female_Black 14
Male_Black 0
Male_Black 1
Male_Black 2
Male_Black 3
Male_Black 4
Male_Black 5
Male_Black 6
Male_Black 7
Male_Black 8
Male_Black 9
Male_Black 10
Male_Black 11
Male_Black 12
Male_Black 13
Male_Black 14
Female_Hispanic 0
Fe

Female_Hispanic 13
Female_Hispanic 14
Male_Hispanic 0
Male_Hispanic 1
Male_Hispanic 2
Male_Hispanic 3
Male_Hispanic 4
Male_Hispanic 5
Male_Hispanic 6
Male_Hispanic 7
Male_Hispanic 8
Male_Hispanic 9
Male_Hispanic 10
Male_Hispanic 11
Male_Hispanic 12
Male_Hispanic 13
Male_Hispanic 14
Female_Asian 0
Female_Asian 1
Female_Asian 2
Female_Asian 3
Female_Asian 4
Female_Asian 5
Female_Asian 6
Female_Asian 7
Female_Asian 8
Female_Asian 9
Female_Asian 10
Female_Asian 11
Female_Asian 12
Female_Asian 13
Female_Asian 14
Male_Asian 0
Male_Asian 1
Male_Asian 2
Male_Asian 3
Male_Asian 4
Male_Asian 5
Male_Asian 6
Male_Asian 7
Male_Asian 8
Male_Asian 9
Male_Asian 10
Male_Asian 11
Male_Asian 12
Male_Asian 13
Male_Asian 14
Female_Caucasian 0
Female_Caucasian 1
Female_Caucasian 2
Female_Caucasian 3
Female_Caucasian 4
Female_Caucasian 5
Female_Caucasian 6
Female_Caucasian 7
Female_Caucasian 8
Female_Caucasian 9
Female_Caucasian 10
Female_Caucasian 11
Female_Caucasian 12
Female_Caucasian 13
Female_Caucasian 

In [125]:
for c in range(4):
    md = pickle.load(open(save_dir+'Dyspnea_case_'+str(c)+'_matched_DDx.pkl', 'rb'))
    DDx_real = cases.loc[c,'DDx'].replace('\n',' ')
    md['Real_DDx'] = DDx_real
    with open(save_dir+'Dyspnea_case_'+str(c)+'_matched_DDx.pkl', 'wb') as file:
        pickle.dump(md, file)